<a href="https://colab.research.google.com/github/vitimc2/audio-dashboard/blob/main/audio%3B_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg


Reading package lists... Done
Building dependency tree       
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1).
libportaudiocpp0 is already the newest version (19.6.0-1).
portaudio19-dev is already the newest version (19.6.0-1).
libasound2-dev is already the newest version (1.1.3-5ubuntu0.6).
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.


In [2]:
!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install PyAudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install SpeechRecognition


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install gTTS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Faz o download do banco de dados

In [6]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1VPmgT3hvHtQxxcLqWA4l2Hdwa6xLOC46' -O /content/ey_dio.db
!chmod 777 /content/ey_dio.db

--2022-10-24 01:54:17--  https://docs.google.com/uc?export=download&id=1VPmgT3hvHtQxxcLqWA4l2Hdwa6xLOC46
Resolving docs.google.com (docs.google.com)... 142.251.16.102, 142.251.16.138, 142.251.16.100, ...
Connecting to docs.google.com (docs.google.com)|142.251.16.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-bc-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/kbo6bhq0dbc09ea5f52su0dmvpi0lovl/1666576425000/06981639974565349704/*/1VPmgT3hvHtQxxcLqWA4l2Hdwa6xLOC46?e=download&uuid=ffe67018-b64a-4ef4-a2ed-029e06fad15b [following]
--2022-10-24 01:54:18--  https://doc-04-bc-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/kbo6bhq0dbc09ea5f52su0dmvpi0lovl/1666576425000/06981639974565349704/*/1VPmgT3hvHtQxxcLqWA4l2Hdwa6xLOC46?e=download&uuid=ffe67018-b64a-4ef4-a2ed-029e06fad15b
Resolving doc-04-bc-docs.googleusercontent.com (doc-04-bc-docs.googleusercontent.com)... 142.251.45.97, 2607:f8b0:4

Importa o SQLite e criar as funções para fazer as consultas

In [7]:
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)

        return conn
    except Error as e:
        print(e)

dbConnection = create_connection("/content/ey_dio.db")

def get_total_vendas(conn):
    c = conn.cursor()
    c.execute("select sum(total) as total from pedidos")
    row = c.fetchone()
    return row[0]

def get_valo_medio_de_venda(conn):
    c = conn.cursor()
    c.execute("select avg(total) as total from pedidos")
    row = c.fetchone()
    return row[0]

def get_produto_mais_vendido(conn):
    c = conn.cursor()
    c.execute("""select pr.descricao, count(*) as qt_vendida, sum(it.total) as vr_total
                 from pedidos p
                   inner join pedido_itens it on it.id_pedido = p.id
                   inner join produtos pr on pr.id = it.id_produto
                 group by pr.descricao  
                 order by 2 desc  
                 limit 1 """)
    row = c.fetchone()
    return {
      "produto": row[0],
      "qt_vendida": row[1],
      "vr_total": row[2]}

def get_cliente_mais_compras(conn):
    c = conn.cursor()
    c.execute("""select c.nome, count(*) as qt_vendas, sum(p.total) as vr_total
                 from pedidos p
                   inner join clientes c on c.id = p.id_cliente
                 group by c.nome
                 order by 2 desc  
                 limit 1
                 """)
    row = c.fetchone()
    return {
      "cliente": row[0],
      "qt_vendas": row[1],
      "vr_total": row[2]}

def get_resumo_vendas(conn):
    c = conn.cursor()
    c.execute("""select count(*) as qt_vendas,
                        sum(p.total) as vr_total,
                        avg(p.total) as vr_medio,
                        min(p.total) as menor_pedido,
                        max(p.total) as maior_pedido
                 from pedidos p
                 """)
    row = c.fetchone()
    return {
      "qt_vendas": row[0],
      "vr_total": row[1],
      "vr_medio": row[2],
      "menor_pedido": row[3],
      "maior_pedido": row[4]}

Código necessário para a captura do audio em JS
Cria a função para gravar o audio e retornar para o colab

In [8]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import io
import ffmpeg
import time

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };
  recorder.start();
  };

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);
recordButton.innerText = "Aguardando comando... click aqui para processar o comando";

function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Aguarde comando em processamento...."
  }
}

function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
  recordButton.onclick = ()=>{
    toggleRecording()

    sleep(2000).then(() => {
      resolve(base64data.toString())
    });

  }
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  time.sleep(1)

  return output

Cria a função que vai "falar" as respotas para os comandos solicitados para o dashboard

In [9]:
import uuid
from gtts import gTTS

def responder(resposta: str):
  language = "pt"
  gtts_object = gTTS(text = resposta, 
                    lang = language,
                    slow = False)
  fileName = "/tmp/"+str(uuid.uuid4())+".wav"
  gtts_object.save(fileName)
  return fileName

In [10]:
import locale
def moeda(valor):
    locale.setlocale(locale.LC_MONETARY, 'en_US.UTF-8')
    valor = locale.currency(valor, grouping=True, symbol=None)
    return valor


Cria a  função para avaliar o comando e executa o código de reconhecimento e dar a informação solicitada pelo cliente.

In [14]:
import speech_recognition as sr

#Função para identificar o comando de voz
def avaliar_comando(wav: any):
    #Habilita o microfone do usuário
    recognizer = sr.Recognizer()
        
    try:
        #cria um AudioDAta com um WAV
        audio = sr.AudioData(wav, 44000, 2)
        
        #Passa a variável para o algoritmo reconhecedor de padroes
        frase = recognizer.recognize_google(audio,language='pt-BR')

        #Retorna a frase pronunciada
        print("Você disse: " + frase)

        if "total" in frase and "vendas" in frase:
            return "O valor total das vendas foi de R$" + moeda(get_total_vendas(dbConnection))

        if "valor" in frase and ("medio" in frase or "médio" in frase):
            return "O valor médio das vendas foi de R$" + moeda(get_valo_medio_de_venda(dbConnection))

        if "produto" in frase and ("vendido" in frase or "vendas" in frase):
            resProd = get_produto_mais_vendido(dbConnection)
            return "O produto mais vendido foi " + resProd["produto"] + " com " + str(resProd["qt_vendida"]) + " unidades vendidas, somando um valor de R$" + moeda(resProd["vr_total"])

        if "cliente" in frase and ("comprou" in frase or "compra" in frase or "melhor" in frase):
            resCli = get_cliente_mais_compras(dbConnection)
            return "O cliente que mais comprou foi " + resCli["cliente"] + " com " + str(resCli["qt_vendas"]) + " compras realizadas, somando um total de R$" + moeda(resCli["vr_total"])

        if "resumo" in frase and "vendas" in frase:
            resVendas = get_resumo_vendas(dbConnection)
            return "Foram realizadas "+str(resVendas["qt_vendas"])+" vendas totalizando R$"+ moeda(resVendas["vr_total"]) + ", o valor médio das vendas foi de R$"+moeda(resVendas["vr_medio"])+", o maior valor de uma venda foi de R$"+moeda(resVendas["maior_pedido"])+" e o menor valor de uma venda R$"+moeda(resVendas["menor_pedido"])

        
    #Se nao reconheceu o padrao de fala, exibe a mensagem
    except sr.UnknownValueError:
        return "Ocorreu um erro inesperado ao processar a solicitação"
        
    return "Não entendi"

audio = get_audio()

resposta = avaliar_comando(audio)

audioResposta = responder(resposta)

Audio(audioResposta, autoplay=True)


Você disse: saber qual foi o produto mais vendido
